In [1]:
from IPython.core.interactiveshell import InteractiveShell

# Display outputs from all lines when they exist
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd

pd.set_option('max_columns', None)

In [3]:
# Read from CSV
# posts_2018_df = pd.read_csv("2018_posts.csv")
# comms_2018_df = pd.read_csv("2018_comments.csv")
# posts_2019_df = pd.read_pickle("2019_posts.csv")
# comms_2019_df = pd.read_pickle("2019_comments.csv")

# Read from PKL
posts_2018_df = pd.read_pickle("2018_posts.pkl")
comms_2018_df = pd.read_pickle("2018_comments.pkl")
posts_2019_df = pd.read_pickle("2019_posts.pkl")
comms_2019_df = pd.read_pickle("2019_comments_partial.pkl")

print(posts_2018_df.shape, comms_2018_df.shape)
print(posts_2019_df.shape, comms_2019_df.shape)

(17598, 132) (448947, 40)
(244595, 95) (1716501, 46)


In [4]:
posts_df = posts_2018_df.append(posts_2019_df).reset_index(drop=True)
comms_df = comms_2018_df.append(comms_2019_df).reset_index(drop=True)

print(posts_df.shape)
print(comms_df.shape)

(262193, 146)
(2165448, 48)


In [5]:
posts_df[posts_df['id']=='9zgc4m'] # the ID of the first bot implementation announcement

# only include posts AFTER that announcement cutoff
posts_df = posts_df[posts_df['created_utc'] > posts_df[posts_df['id']=='9zgc4m'].iloc[0]['created_utc']].reset_index(drop=True)
print(posts_df.shape)
# and cut down the comments df to only include data for posts within that dataset, while we're at it
comms_df = comms_df[comms_df['link_id'].apply(lambda x: x[3:]).isin(posts_df['id'])].reset_index(drop=True)
print(comms_df.shape)

,author,author_flair_css_class,author_flair_text,brand_safe,can_mod_post,contest_mode,created_utc,domain,full_link,id,is_crosspostable,is_reddit_media_domain,is_self,is_video,locked,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,pinned,retrieved_on,score,selftext,spoiler,stickied,subreddit,subreddit_id,subreddit_type,suggested_sort,thumbnail,title,url,whitelist_status,thumbnail_height,thumbnail_width,link_flair_text,banned_by,edited,media.oembed.description,media.oembed.height,media.oembed.html,media.oembed.provider_name,media.oembed.provider_url,media.oembed.thumbnail_height,media.oembed.thumbnail_url,media.oembed.thumbnail_width,media.oembed.title,media.oembed.type,media.oembed.version,media.oembed.width,media.type,media_embed.content,media_embed.height,media_embed.scrolling,media_embed.width,secure_media.oembed.description,secure_media.oembed.height,secure_media.oembed.html,secure_media.oembed.provider_name,secure_media.oembed.provider_url,secure_media.oembed.thumbnail_height,secure_media.oembed.thumbnail_url,secure_media.oembed.thumbnail_width,secure_media.oembed.title,secure_media.oembed.type,secure_media.oembed.version,secure_media.oembed.width,secure_media.type,secure_media_embed.content,secure_media_embed.height,secure_media_embed.media_domain_url,secure_media_embed.scrolling,secure_media_embed.width,link_flair_css_class,post_hint,preview.enabled,preview.images,author_flair_richtext,author_flair_type,link_flair_richtext,link_flair_text_color,link_flair_type,rte_mode,author_flair_background_color,author_flair_text_color,media.oembed.author_name,media.oembed.author_url,secure_media.oembed.author_name,secure_media.oembed.author_url,link_flair_background_color,link_flair_template_id,author_cakeday,gilded,send_replies,no_follow,subreddit_subscribers,crosspost_parent,crosspost_parent_list,is_original_content,wls,pwls,media_only,author_id,is_meta,archived,author_created_utc,author_flair_template_id,author_fullname,author_patreon_flair,can_gild,category,content_categories,distinguished,hidden,is_robot_indexable,media,quarantine,removal_reason,secure_media,subreddit_name_prefixed,gildings.gid_1,gildings.gid_2,gildings.gid_3,all_awardings,allow_live_comments,awarders,total_awards_received,treatment_tags,upvote_ratio,author_premium,removed_by_category,updated_utc,preview.reddit_video_preview.dash_url,preview.reddit_video_preview.duration,preview.reddit_video_preview.fallback_url,preview.reddit_video_preview.height,preview.reddit_video_preview.hls_url,preview.reddit_video_preview.is_gif,preview.reddit_video_preview.scrubber_media_url,preview.reddit_video_preview.transcoding_status,preview.reddit_video_preview.width,steward_reports,og_description,og_title,removed_by
10541,TheOutrageousClaire,badge,Colo-rectal Surgeon,NaN,False,False,1542909710,self.AmItheAsshole,https://www.reddit.com/r/AmItheAsshole/comment...,9zgc4m,True,False,True,False,False,104,0,False,house_only,/r/AmItheAsshole/comments/9zgc4m/change_is_com...,False,1542909711,1,"We've got 200,000 subscribers now and it feels...",False,False,AmItheAsshole,t5_2xhvq,public,NaN,self,Change is coming....,https://www.reddit.com/r/AmItheAsshole/comment...,house_only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],text,[],dark,text,NaN,,dark,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,True,True,200579.0,NaN,NaN,False,1.0,1.0,False,NaN,False,NaN,NaN,NaN,t2_7j6x2,False,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(251651, 146)
(1913808, 48)


In [6]:
# Create list of users that should always be filtered out from the counts
restricted_users_list = ['AutoModerator', 'Judgement_Bot_AITA']

In [7]:
posts_df[:2]

,author,author_flair_css_class,author_flair_text,brand_safe,can_mod_post,contest_mode,created_utc,domain,full_link,id,is_crosspostable,is_reddit_media_domain,is_self,is_video,locked,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,pinned,retrieved_on,score,selftext,spoiler,stickied,subreddit,subreddit_id,subreddit_type,suggested_sort,thumbnail,title,url,whitelist_status,thumbnail_height,thumbnail_width,link_flair_text,banned_by,edited,media.oembed.description,media.oembed.height,media.oembed.html,media.oembed.provider_name,media.oembed.provider_url,media.oembed.thumbnail_height,media.oembed.thumbnail_url,media.oembed.thumbnail_width,media.oembed.title,media.oembed.type,media.oembed.version,media.oembed.width,media.type,media_embed.content,media_embed.height,media_embed.scrolling,media_embed.width,secure_media.oembed.description,secure_media.oembed.height,secure_media.oembed.html,secure_media.oembed.provider_name,secure_media.oembed.provider_url,secure_media.oembed.thumbnail_height,secure_media.oembed.thumbnail_url,secure_media.oembed.thumbnail_width,secure_media.oembed.title,secure_media.oembed.type,secure_media.oembed.version,secure_media.oembed.width,secure_media.type,secure_media_embed.content,secure_media_embed.height,secure_media_embed.media_domain_url,secure_media_embed.scrolling,secure_media_embed.width,link_flair_css_class,post_hint,preview.enabled,preview.images,author_flair_richtext,author_flair_type,link_flair_richtext,link_flair_text_color,link_flair_type,rte_mode,author_flair_background_color,author_flair_text_color,media.oembed.author_name,media.oembed.author_url,secure_media.oembed.author_name,secure_media.oembed.author_url,link_flair_background_color,link_flair_template_id,author_cakeday,gilded,send_replies,no_follow,subreddit_subscribers,crosspost_parent,crosspost_parent_list,is_original_content,wls,pwls,media_only,author_id,is_meta,archived,author_created_utc,author_flair_template_id,author_fullname,author_patreon_flair,can_gild,category,content_categories,distinguished,hidden,is_robot_indexable,media,quarantine,removal_reason,secure_media,subreddit_name_prefixed,gildings.gid_1,gildings.gid_2,gildings.gid_3,all_awardings,allow_live_comments,awarders,total_awards_received,treatment_tags,upvote_ratio,author_premium,removed_by_category,updated_utc,preview.reddit_video_preview.dash_url,preview.reddit_video_preview.duration,preview.reddit_video_preview.fallback_url,preview.reddit_video_preview.height,preview.reddit_video_preview.hls_url,preview.reddit_video_preview.is_gif,preview.reddit_video_preview.scrubber_media_url,preview.reddit_video_preview.transcoding_status,preview.reddit_video_preview.width,steward_reports,og_description,og_title,removed_by
0,hshsvdhsiabs,None,None,NaN,False,False,1542909884,self.AmItheAsshole,https://www.reddit.com/r/AmItheAsshole/comment...,9zgd5n,True,False,True,False,False,10,0,False,house_only,/r/AmItheAsshole/comments/9zgd5n/wibta_for_not...,False,1542909885,1,I had this best friend in high school and we b...,False,False,AmItheAsshole,t5_2xhvq,public,NaN,self,WIBTA for not responding to an ex friends nice...,https://www.reddit.com/r/AmItheAsshole/comment...,house_only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],text,[],dark,text,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,True,True,200588.0,NaN,NaN,False,1.0,1.0,False,NaN,False,NaN,NaN,NaN,t2_1m2ny8mg,False,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ilikeallburgers,None,None,NaN,False,False,1542910040,self.AmItheAsshole,https://www.reddit.com/r/AmItheAsshole/comment...,9zge0j,True,False,True,False,False,8,0,False,house_only,/r/AmItheAsshole/comments/9zge0j/aita_for_not_...,False,1542910040,1,My cousin and her fiance are getting married i...,False,False,AmItheAsshole,t5_2xhvq,publ

In [8]:
comms_df[:2]

,approved_at_utc,author,author_flair_css_class,author_flair_text,banned_at_utc,body,can_mod_post,collapsed,collapsed_reason,created_utc,distinguished,edited,id,is_submitter,link_id,parent_id,permalink,retrieved_on,score,stickied,subreddit,subreddit_id,author_cakeday,author_flair_background_color,author_flair_richtext,author_flair_text_color,author_flair_type,no_follow,rte_mode,send_replies,author_flair_template_id,author_id,author_fullname,gildings.gid_1,gildings.gid_2,gildings.gid_3,author_patreon_flair,all_awardings,locked,total_awards_received,associated_award,author_premium,awarders,collapsed_because_crowd_control,comment_type,top_awarded_type,treatment_tags,steward_reports
0,None,AutoModerator,None,None,None,^^^^AUTOMOD ***This is a copy of the above po...,False,False,None,1542909885,moderator,False,ea8xh7v,False,t3_9zgd5n,t3_9zgd5n,/r/AmItheAsshole/comments/9zgd5n/wibta_for_not...,1542909886,1,False,AmItheAsshole,t5_2xhvq,NaN,None,[],None,text,True,NaN,False,None,NaN,t2_6l4z3,0.0,0.0,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,ursa-majcr,None,None,None,"NTA. you don’t owe her anything, certainly not...",False,False,None,1542910265,None,False,ea8xxe0,False,t3_9zgd5n,t3_9zgd5n,/r/AmItheAsshole/comments/9zgd5n/wibta_for_not...,1542910266,1,False,AmItheAsshole,t5_2xhvq,NaN,None,[],None,text,True,NaN,True,None,NaN,t2_17d1fuhb,0.0,0.0,0.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Get the judgement that a specific comment gives, if any
def comment_judgement(comms_df):
    # Judgement tags are in {'NTA', 'YTA', 'NAH', 'ESH', 'INFO', 'SHP' (not formal after a certain point but still commonly quoted)}
    potential_tags = ['NTA', 'YTA', 'NAH', 'ESH', 'INFO', 'SHP']
    tags = {'id':[], 'judgement_tag':[]}
    for _, comment in comms_df.iterrows():
        judgements = []
        for ptag in potential_tags:
            if ptag in comment['body']:
                judgements.append(ptag)
        tags['id'].append(comment['id'])
        if len(judgements)==1:
            tags['judgement_tag'].append(judgements[0])
        else:
            tags['judgement_tag'].append(None)
    return pd.DataFrame(tags)

comment_judgement(comms_df[:5])
# comms_df[:5].merge(comment_judgement(comms_df[:5]), left_on='id', right_on='id')

,id,judgement_tag
0,ea8xh7v,None
1,ea8xxe0,NTA
2,ea8xzeb,NTA
3,ea8ymxv,NTA
4,ea8yydr,NTA


In [ ]:
# Parse all comments and pick out their judgement tags for ease of use later
comms_df.merge(comment_judgement(comms_df), left_on='id', right_on='id')
comms_df.to_pickle("backup_comms_df.pkl")

# comms_df = pd.read_pickle("backup_comms_df.pkl")

In [21]:
# Get the final post judgement based on top-scoring comment with a judgement
def tag_post_judgements(posts_df, comms_df):
    # VAL1: Return mapping between t3-less post ID to several attributes related to the judgements of a post
    # ATTR: Final judgement tag: determined by top-scoring comment with a judgement tag
    # ATTR: Controversialness metric: based on what r/AITAFiltered uses, the ratio between judgements of (ESH/YTA) : (NAH/NTA)
    # Let X be the number of ESH/YTA ("you're an asshole") judgements
    # Let Y be the number of NAH/NTA ("you're not an asshole") judgements
    # Then X/(X+Y)=Z is the fraction of all decided (non-SHP and non-INFO) judgements that call the OP an asshole
    # Then calculating 1+(-2)(|Z-0.5|) is a linear metric in the range [0,1], where 0 means full consensus and 1 means 50-50 judgement split
    # If there are no judgements, then the controversialness is 0 by default
    # ATTR: Judgement distribution: a tuple containing counts for (NTA, YTA, NAH, ESH, INFO) judgements
    # VAL2: Return mapping between t3-less post ID with individual usernames, the judgements they left on that post, and the comment score they had for that judgement
    tags = {'id':[], 'final_judgement':[], 'controversialness':[], 'controversialness_distrib':[]}
    userdistrib = {'id':[], 'judgement_username':[], 'judgement_decision':[], 'judgement_score':[], 'judgement_correct':[]}
    for _, post in posts_df.iterrows():
        post_id = post['id']
        tags['id'].append(post_id)
        # get the judgement
        judgements = comms_df[comms_df['link_id']==f't3_{post_id}']
        # Filter out comments from bots, OP
        judgements = judgements[judgements['author'] != post['author']]
        judgements = judgements[~ judgements['author'].isin(restricted_users_list)]
        if len(judgements)==0:
            tags['final_judgement'].append(None)
            tags['controversialness'].append(0)
            tags['controversialness_distrib'].append( (0, 0, 0, 0, 0) )
            continue
        # Get the judgement tags for all relevant comments if they're not already processed
        if 'judgement_tag' not in judgements:
            judgements = judgements.merge(comment_judgement(judgements), left_on='id', right_on='id')
        # Filter out comments without any judgement tag
        judgements = judgements[judgements['judgement_tag'].apply(lambda x: x is not None)]
        if len(judgements)==0:
            tags['final_judgement'].append(None)
            tags['controversialness'].append(0)
            tags['controversialness_distrib'].append( (0, 0, 0, 0, 0) )
            continue
        # Sort comments by score, descending. If there are tied scores, prioritize the one posted earlier
        judgements = judgements.sort_values(by=['score', 'created_utc'], ascending=[False, True])
        post_final_judgement = judgements.iloc[0]['judgement_tag']
        # Save final judgement attributes
        tags['final_judgement'].append(post_final_judgement)
        # Read through each judgement and collect user mapping, indiv judgement type counts
        users_parsed = set()
        judgement_counts = {'NTA':0, 'YTA':0, 'NAH':0, 'ESH':0, 'INFO':0}
        for _, x in judgements.iterrows():
            # get user mapping
            username = x['author']
            judgement = x['judgement_tag']
            if username not in users_parsed:
                userdistrib['id'].append(post_id)
                userdistrib['judgement_username'].append(username)
                userdistrib['judgement_decision'].append(judgement)
                userdistrib['judgement_score'].append(x['score'])
                userdistrib['judgement_correct'].append(1 if (judgement==post_final_judgement) else 0)
                users_parsed.add(username)
            # get the judgement type
            if judgement in judgement_counts:
                judgement_counts[judgement] += 1
        # Calculate controversialness metric
        controversialness = (judgement_counts['ESH']+judgement_counts['YTA'])/(judgement_counts['ESH']+judgement_counts['YTA']+judgement_counts['NAH']+judgement_counts['NTA'])
        controversialness = abs(controversialness - 0.5)
        controversialness = controversialness * -2
        controversialness = controversialness + 1
        # Save judgement distribution attributes
        tags['controversialness'].append(controversialness)
        tags['controversialness_distrib'].append((
            judgement_counts['NTA'], 
            judgement_counts['YTA'], 
            judgement_counts['NAH'], 
            judgement_counts['ESH'], 
            judgement_counts['INFO']))
    return pd.DataFrame(tags), pd.DataFrame(userdistrib)

temp_a, temp_b = tag_post_judgements(posts_df[:5], comms_df)
temp_a
temp_b[:5]
# posts_df[:5].merge(tag_post_judgements(posts_df[:5], comms_df), left_on='id', right_on='id')

,id,final_judgement,controversialness,controversialness_distrib
0,9zgd5n,NTA,0.000000,"(6, 0, 0, 0, 0)"
1,9zge0j,NTA,0.333333,"(5, 1, 0, 0, 0)"
2,9zgikj,YTA,0.800000,"(3, 2, 0, 0, 0)"
3,9zgirj,None,0.000000,"(0, 0, 0, 0, 0)"
4,9zglje,YTA,0.555556,"(10, 26, 0, 0, 0)"


,id,judgement_username,judgement_decision,judgement_score,judgement_correct
0,9zgd5n,ursa-majcr,NTA,1,1
1,9zgd5n,LikeAGreenBean,NTA,1,1
2,9zgd5n,cruelcherry,NTA,1,1
3,9zgd5n,_Liv__,NTA,1,1
4,9zgd5n,LampsPlus1,NTA,1,1


In [12]:
# TODO implement relationship between user and final judgement
# TODO implement sentiment difference between a pair of comments

In [13]:
import networkx as nx

gAll = {}

for _, post in posts_df.iterrows():
    g = nx.DiGraph()
    post_id = post['id']
    userResponders = {}
    for _, comment in comms_df[comms_df['link_id']==f't3_{post_id}'].iterrows():
        parentID = comment['parent_id']
        currentID = 't1_'+comment['id']
        author = comment['author']
        g.add_edge(parentID, currentID)
        g.nodes[currentID]['author'] = author
        if author not in userResponders:
            userResponders[author] = []
        userResponders[author].append(currentID)    
    gAll[f't3_{post_id}'] = {
        'post_graph': g,
        'user_op': post['author'],
        'user_responders': userResponders,
    }

KeyboardInterrupt: 

In [ ]:
temp_max = ''
maxcount = 0
for _, post in posts_df.iloc[:100].iterrows():
    commcount = post['num_comments']
    if commcount > maxcount:
        maxcount = commcount
        temp_max = 't3_'+post['id']
print(temp_max)

In [ ]:
# test out the graph building
postID = 't3_9zigtr' # list(gAll.keys())[9999]
postG = gAll[postID]['post_graph']
# print(postG.nodes.data())
nx.draw_kamada_kawai(
    postG, 
#     node_color=[
#         d['comment_author'] if 'comment_author' in d else gAll[postID]['user_op']
#         for (n,d) in postG.nodes.data()
#     ]
)
print(postID)
print(gAll[postID]['user_op'])
print(gAll[postID]['user_responders'])

In [ ]:
# count how many branches a user has touched
# path metric:
# count of how many unique paths are needed for all of the user's posts to be reached
def branch_count(username, postID):
    # retrieve graph we are working with
    g = gAll[postID]['post_graph']
    # calculate path to each of the poster's comments
    commentPaths = {}
    for cID in gAll[postID]['user_responders'][username]:
        try:
            commentPaths[cID] = nx.shortest_path(g, postID, cID)
        except:
            continue
    # starting from longest path, remove all comments along that path (since theyre not unique)
    pathCount = 0
    while len(commentPaths) > 0:
#         print(commentPaths)
        deepestCID = sorted(
            [(k, len(commentPaths[k])) for k in commentPaths],
            key=(lambda x: x[1]),
        )
        deepestPath = commentPaths[deepestCID[-1][0]]
        for cID in deepestPath:
            commentPaths.pop(cID, None)
        pathCount += 1
    return pathCount

username = "Killairmanable"
postID = "t3_9zigtr"
print(branch_count(username, postID))

In [ ]:
import statistics

# higher = more breadth
def branchiness(username, postID, getGraph=False):
    # retrieve graph we are working with
    g = gAll[postID]['post_graph']
    # calculate path to each of the poster's comments
    commentPaths = {}
    for cID in gAll[postID]['user_responders'][username]:
        try:
            commentPaths[cID] = nx.shortest_path(g, postID, cID)
        finally:
            # print(f'{cID} nonexistent comment?')
            continue
    # starting from shortest path, build out tree of direct dependencies
    gPath = nx.DiGraph()
    nodesAdded = [postID]
    while len(commentPaths) > 0:
#         print(commentPaths)
        closestCID = sorted(
            [(k, len(commentPaths[k])) for k in commentPaths],
            key=(lambda x: x[1]),
        )
        closestPath = commentPaths[closestCID[0][0]]
        closestCID = closestPath[-1]
        topLevel = True
        for n in nodesAdded:
            if nx.has_path(g, n, closestCID):
                topLevel = False
                gPath.add_edge(n, closestCID)
                nodesAdded = [closestCID]+nodesAdded
                break
        if topLevel:
            gPath.add_edge(postID, closestCID)
            nodesAdded = [closestCID]+nodesAdded
        commentPaths.pop(closestCID, None)
    # calculate average degree
    degrees = [d for (n,d) in gPath.out_degree() if d!=0]
    if len(degrees)==0:
        # there was some hole in the graph between ALL of a user's comments and the main post...
        return None, None
    degreeAvg = sum(degrees)/len(degrees)
    degreeMed = statistics.median(degrees)
    if getGraph:
        return degreeAvg, gPath
    return degreeAvg, degreeMed

username = "Killairmanable"
postID = "t3_9zigtr"
print(branchiness(username, postID))

# Sanity check this metric?

import math
import matplotlib.pyplot as plt

t = []
for postID in gAll:
    for uname in gAll[postID]['user_responders']:
        if uname!=gAll[postID]['user_op'] and uname not in re:
            brtuple = branchiness(uname, postID)
            if brtuple[0] is not None:
                t.append((
                    len(gAll[postID]['user_responders'][uname]),
                    brtuple,
                ))

tNext = []
for l in set([i[0] for i in t]):
    v = [i[1][0] for i in t if i[0]==l]
    v = sum(v)/len(v)
    tNext.append( (l, v) )
plt.scatter(
    [x for (x, _) in tNext], 
    [y for (_, y) in tNext], 
)
plt.xlabel('# of comments')
plt.ylabel('avg(avg degree of indiv comment tree)')
plt.show()

tNext = []
for l in set([i[0] for i in t]):
    v = [i[1][1] for i in t if i[0]==l]
    v = sum(v)/len(v)
    tNext.append( (l, v) )
plt.scatter(
    [x for (x, _) in tNext], 
    [y for (_, y) in tNext], 
)
plt.xlabel('# of comments')
plt.ylabel('avg(med degree of indiv comment tree)')
plt.show()

In [ ]:
# get branch count for responders
distribCount = {} # map from [number of branches]: [users who have had that number]
userCount = {}    # map from [usernames]: [branch distribution they've done]
userBCount = {}    # map from [usernames]: [branchiness distribution they've done]
for postID in gAll.keys():
    for u in gAll[postID]['user_responders']:
        if u!=gAll[postID]['user_op'] and u not in restricted_users_list:
            count = branch_count(u, postID)
            if count!=0:
                ctrlcount = branchiness(u, postID)
                if count not in distribCount:
                    distribCount[count] = []
                distribCount[count].append(u)
                if u not in userCount:
                    userCount[u] = []
                userCount[u].append(count)
                if u not in userBCount:
                    userBCount[u] = []
                userBCount[u].append(ctrlcount)

In [ ]:
import math
import matplotlib.pyplot as plt

t = [(k, len(set(distribCount[k])), set(distribCount[k])) for k in distribCount.keys()]
t = [(a,b,c) for (a,b,c) in t if b!=0]
t = sorted(t, key=lambda x: x[0])
print([(a,b) for (a,b,c) in t])

plt.scatter(
    [b for (b, _, _) in t], 
    [math.log(fq) for (_, fq, _) in t], 
)
plt.xlabel('# of branches (X)')
plt.ylabel('log(# of users who touched X branches)')
plt.show()

In [ ]:
import math
import matplotlib.pyplot as plt

t = [(k, sum(userCount[k]), len(userCount[k]), max(userCount[k])) for k in userCount.keys()]
t = sorted(t, key=lambda x: x[2], reverse=True)
print('(username, avg branches/post, max branches/post, #posts touched)')
for i in ([(a,b/c,d,c) for (a,b,c,d) in t])[:10]:
    print(i)

plt.scatter(
    [pc for (_, _, pc, _) in t], 
    [math.log(sumB/numB) for (_, sumB, numB, _) in t],
    alpha=0.05
)
plt.xlabel('# of posts commented on')
plt.ylabel('log(Average number of branches touched)')
plt.show()

plt.scatter(
    [pc for (_, _, pc, _) in t], 
    [maxB for (_, _, _, maxB) in t],
    alpha=0.05
)
plt.xlabel('# of posts commented on')
plt.ylabel('Max number of branches touched')
plt.show()

In [ ]:
import math
import matplotlib.pyplot as plt

t = [(k, sum([e[0] for e in userBCount[k]]), len([e[0] for e in userBCount[k]]), max([e[0] for e in userBCount[k]])) for k in userBCount.keys()]
t = sorted(t, key=lambda x: x[2], reverse=True)
print('(username, avg branchiness/post, max branchiness/post, #posts touched)')
for i in ([(a,b/c,d,c) for (a,b,c,d) in t])[:10]:
    print(i)

plt.scatter(
    [pc for (_, _, pc, _) in t], 
    [math.log(sumB/numB) for (_, sumB, numB, _) in t],
    alpha=0.05
)
plt.xlabel('# of posts commented on')
plt.ylabel('log(Average branchiness)')
plt.show()

plt.scatter(
    [pc for (_, _, pc, _) in t], 
    [maxB for (_, _, _, maxB) in t],
    alpha=0.05
)
plt.xlabel('# of posts commented on')
plt.ylabel('Max branchiness')
plt.show()

In [ ]:
import math
import matplotlib.pyplot as plt

t = [[e[0] for e in userBCount[k]] for k in userBCount.keys()]
t = [i for sl in t for i in sl]
plt.hist(t, bins=100)
plt.xlabel('# of occurrences')
plt.ylabel('Branchiness (per user, per post)')
plt.show()

t = [[e[0] for e in userBCount[k]] for k in userBCount.keys()]
t = [i for sl in t for i in sl]
t = [i for i in t if i<5]
plt.hist(t, bins=100)
plt.xlabel('# of occurrences')
plt.ylabel('Branchiness (per user, per post) range [0,5)')
plt.show()

t = [[e[0] for e in userBCount[k]] for k in userBCount.keys()]
t = [i for sl in t for i in sl]
t = [i for i in t if i<5 and i>1]
plt.hist(t, bins=100)
plt.xlabel('# of occurrences')
plt.ylabel('Branchiness (per user, per post) range (1,5)')
plt.show()